In [10]:
import numpy as np
from matplotlib import pyplot as plt

# function definitions

def load_data(fname):
    data = np.genfromtxt(fname, delimiter=',', dtype=str)

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i][j] == '?':
                data[i][j] = np.random.choice(['n','y'])
    
    return data


def ID3(d, n, data):
    # you will probably need additional helper functions
    return tree

def compute_gain(S, i):
    
    return gain


def split_data(data, split):
    
    return training_set, test_set


def learning_curve(d, n, training_set, test_set):
    # you will probably need additional helper functions
    return plot

# main

data = load_data('house-votes-84.data')
